In [1]:
import numpy as np
import pandas as pd
import pickle


# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix

# # sklearn imports
# from sklearn.pipeline import make_pipeline
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, accuracy_score, f1_score, balanced_accuracy_score

# # tools for plotting confusion matrices
# import matplotlib
# from matplotlib import pyplot as plt
# import seaborn as sn

In [2]:
def load_dataset_ctrvsone_and_transfer(user_split = True, runidx=1, data_type='meditation', one_class='HT'):
    X_train = None
    y_train_medid = None
    y_train_subjid = None
    X_val = None
    y_val_medid = None
    y_val_subjid = None
    X_test = None
    y_test_medid = None
    y_test_subjid = None

    dtype_names = ["HT", "SNY", "VIP", "CTR"]

    # user_split: Determines the creation of train/val/test set. In user_split, test/val users are never seen during the train. time_split randomly splits each user's chunks into train/val/test.
    if user_split:
        data_file_path = '../../iconip_data/{}/user_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)
    else:
        data_file_path = '../../iconip_data/{}/time_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)

    with open(data_file_path, 'rb') as f:
        all_data_splits = pickle.load(f)

        data_index_label_oneclass = dtype_names.index(one_class)
        data_index_label_ctr = dtype_names.index('CTR')
        transfer_index_labels = [x for x in list(set([0,1,2,3]).difference(set([dtype_names.index(one_class), 3]))) ]
        
        X_train = all_data_splits['train']['x']
        y_train_medid = all_data_splits['train']['y_med']
        y_train_subjid = all_data_splits['train']['y_subj']
        y_train_ts = all_data_splits['train']['y_ts']
        clf_config_subset_indices = np.where((y_train_medid == data_index_label_oneclass) | (y_train_medid == data_index_label_ctr))[0]
        X_train = X_train[(clf_config_subset_indices)]
        y_train_medid = y_train_medid[(clf_config_subset_indices)]
        y_train_subjid = y_train_subjid[clf_config_subset_indices]
        y_train_ts = y_train_ts[(clf_config_subset_indices)]

        X_val = all_data_splits['val']['x']
        y_val_medid = all_data_splits['val']['y_med']
        y_val_subjid = all_data_splits['val']['y_subj']
        y_val_ts = all_data_splits['val']['y_ts']
        clf_config_subset_indices = np.where((y_val_medid == data_index_label_oneclass) | (y_val_medid == data_index_label_ctr))[0]
        X_val = X_val[(clf_config_subset_indices)]
        y_val_medid = y_val_medid[(clf_config_subset_indices)]
        y_val_subjid = y_val_subjid[(clf_config_subset_indices)]
        y_val_ts = y_val_ts[(clf_config_subset_indices)]
        
        # Pretrain test and transfer test set
        X_test = all_data_splits['test']['x']
        y_test_medid = all_data_splits['test']['y_med']
        y_test_subjid = all_data_splits['test']['y_subj']
        y_test_ts = all_data_splits['test']['y_ts']
        
        # transfer test contains CTR from test and other two classes from the test.
        ctr_othertwo_indices = np.where((y_test_medid == data_index_label_ctr) | (y_test_medid == transfer_index_labels[0]) | (y_test_medid == transfer_index_labels[1]))[0]
        X_test_transfer = X_test[(ctr_othertwo_indices)]
        y_test_transfer_medid = y_test_medid[(ctr_othertwo_indices)]
        y_test_transfer_subjid = y_test_subjid[(ctr_othertwo_indices)]
        
        # pretrain test filter CTR and one_class from the test.
        clf_config_subset_indices = np.where((y_test_medid == data_index_label_oneclass) | (y_test_medid == data_index_label_ctr))[0]
        X_test = X_test[(clf_config_subset_indices)]
        y_test_medid = y_test_medid[(clf_config_subset_indices)]
        y_test_subjid = y_test_subjid[(clf_config_subset_indices)]
        y_test_ts = y_test_ts[(clf_config_subset_indices)]

    return X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid

In [3]:
def train_model(user_split, med_tech_clf, data_type, one_class):
    
    kernels, chans, samples = 1, 64, 2560
    
    split_strat = 'user' if user_split else 'time'
    model_results = pd.DataFrame()

    for runidx in range(1, 6):
        model_results['{}_Train_Acc'.format(runidx)] = None
        model_results['{}_Val_Acc'.format(runidx)] = None
        model_results['{}_Test_Acc'.format(runidx)] = None
        model_results['{}_Test_Transfer_Acc'.format(runidx)] = None
        model_results['{}_best_params'.format(runidx)] = None


    for runidx in range(1, 6):
        print("RUN ID: ", runidx)
        med_tech_clf = True
        
        X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_transfer, y_test_transfer_medid, y_test_transfer_subjid = load_dataset_ctrvsone_and_transfer(user_split=user_split, runidx=runidx, data_type=data_type, one_class=one_class)
        ## Convert data to binary classification task, i.e. meditation expert as class 0 vs control group as class 1
        y_train_medid = y_train_medid//3
        y_val_medid = y_val_medid//3
        y_test_medid = y_test_medid//3
        y_test_transfer_medid = y_test_transfer_medid//3

        best_clf_ours = None
        best_clf_val = 0

        #X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid
        if med_tech_clf:
            Y_train      = y_train_medid
            Y_validate   = y_val_medid
            Y_test       = y_test_medid
            Y_transfer   = y_test_transfer_medid
            ckpt_file_suffix = '{}_med_clf'.format(split_strat)
        else:
            Y_train      = y_train_subjid
            Y_validate   = y_val_subjid
            Y_test       = y_test_subjid
            Y_transfer   = y_test_transfer_subjid
            ckpt_file_suffix = '{}_subj_clf'.format(split_strat)
        nb_classes = np.unique(Y_train).shape[0]
        print(np.unique(Y_train), ": unique class labels.")
    
        ############################# EEGNet portion ##################################
        # convert labels to one-hot encodings.
        Y_train      = np_utils.to_categorical(Y_train)
        Y_validate   = np_utils.to_categorical(Y_validate)
        Y_test       = np_utils.to_categorical(Y_test)
        Y_transfer   = np_utils.to_categorical(Y_transfer)

        # convert data to NHWC (trials, channels, samples, kernels) format. Data 
        # contains 60 channels and 151 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
        X_val        = X_val.reshape(X_val.shape[0], chans, samples, kernels)
        X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
        X_transfer   = X_transfer.reshape(X_transfer.shape[0], chans, samples, kernels)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_test.shape[0], 'test samples')
        print(X_transfer.shape[0], 'transfer samples')
        print(nb_classes, " number of classes")
    
        # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
        # model configurations may do better, but this is a good starting point)
        model = EEGNet(nb_classes = nb_classes, Chans = chans, Samples = samples, 
                    dropoutRate = 0.5, kernLength = 128, F1 = 8, D = 2, F2 = 16, 
                    dropoutType = 'Dropout')

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', 
                    metrics = ['accuracy'])

        # count number of parameters in the model
        numParams    = model.count_params()    

        # set a valid path for your system to record model checkpoints
        checkpointer = ModelCheckpoint(filepath='./ckpt/checkpoint_{}.h5'.format(ckpt_file_suffix), 
                                    verbose=1, save_best_only=True)
        
        ###############################################################################
        # if the classification task was imbalanced (significantly more trials in one
        # class versus the others) you can assign a weight to each class during 
        # optimization to balance it out. This data is approximately balanced so we 
        # don't need to do this, but is shown here for illustration/completeness. 
        ###############################################################################

        # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
        # the weights all to be 1
        class_weights = {cl: 1 for cl in range(nb_classes)}

        ################################################################################
        # fit the model. Due to very small sample sizes this can get
        # pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
        # Riemannian geometry classification (below)
        ################################################################################
        escallback = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
        fittedModel = model.fit(X_train, Y_train, batch_size = 128, epochs = 10, 
                                verbose = 2, validation_data=(X_val, Y_validate),
                                callbacks=[checkpointer, escallback], class_weight = class_weights)

        # load optimal weights
        model.load_weights('./ckpt/checkpoint_{}.h5'.format(ckpt_file_suffix))
    
        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)

        probs_transfer = model.predict(X_transfer)
        preds_transfer = probs_transfer.argmax(axis = -1)
        
        # save preds on train and val set for error analysis
        preds_train_eegnet = model.predict(X_train)
        preds_train_eegnet = preds_train_eegnet.argmax(axis = -1)
        preds_val_eegnet = model.predict(X_val)
        preds_val_eegnet = preds_val_eegnet.argmax(axis = -1)
        
        
        acc_test         = np.mean(preds == Y_test.argmax(axis=-1))
        acc_transfer     = np.mean(preds_transfer == Y_transfer.argmax(axis=-1))
        print("EEGNet Classification accuracy test and transfer: %f %f" % (acc_test, acc_transfer))
        model_results.loc["EEGNet", ['{}_Train_Acc'.format(runidx), '{}_Val_Acc'.format(runidx), '{}_Test_Acc'.format(runidx), '{}_Test_Transfer_Acc'.format(runidx)]] = [preds_train_eegnet, preds_val_eegnet, acc_test, acc_transfer]
    if user_split:
        model_results.to_csv('./RESULTS_T5{}_MED_TransferPRED_{}_CTRvs{}.csv'.format('USER', data_type, one_class))
    else:
        model_results.to_csv('./RESULTS_T5_MED_TransferPRED_{}_CTRvs{}.csv'.format(data_type, one_class))

    return model_results

In [4]:
# time split
for one_class in ['HT', 'SNY', 'VIP']:
    model_results = train_model(user_split=False, med_tech_clf=True, data_type='meditation', one_class=one_class)

RUN ID:  1
[0 1] : unique class labels.
X_train shape: (990, 64, 2560, 1)
990 train samples
344 test samples
523 transfer samples
2  number of classes


2022-06-23 13:16:04.762971: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10

Epoch 1: val_loss improved from inf to 0.69355, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 156s - loss: 0.7114 - accuracy: 0.5141 - val_loss: 0.6936 - val_accuracy: 0.5589 - 156s/epoch - 19s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.69355 to 0.67995, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 169s - loss: 0.6614 - accuracy: 0.6000 - val_loss: 0.6800 - val_accuracy: 0.5801 - 169s/epoch - 21s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.67995 to 0.64631, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 176s - loss: 0.5986 - accuracy: 0.6848 - val_loss: 0.6463 - val_accuracy: 0.6012 - 176s/epoch - 22s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.64631 to 0.57792, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 173s - loss: 0.4959 - accuracy: 0.7596 - val_loss: 0.5779 - val_accuracy: 0.6405 - 173s/epoch - 22s/step
Epoch 5/10

Epoch 5: val_loss improved from 0.57792 to 0.51775, saving model to ./ckpt/checkpoint_time_m

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (990, 64, 2560, 1)
990 train samples
344 test samples
523 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.68796, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 180s - loss: 0.6990 - accuracy: 0.5333 - val_loss: 0.6880 - val_accuracy: 0.5257 - 180s/epoch - 22s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.68796 to 0.67864, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 174s - loss: 0.6608 - accuracy: 0.6020 - val_loss: 0.6786 - val_accuracy: 0.5468 - 174s/epoch - 22s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.67864 to 0.65528, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 177s - loss: 0.6259 - accuracy: 0.6616 - val_loss: 0.6553 - val_accuracy: 0.5710 - 177s/epoch - 22s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.65528 to 0.61319, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 173s - loss: 0.5628 - accuracy: 0.7152 - val_loss: 0.6132 - val_accur

...

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (996, 64, 2560, 1)
996 train samples
351 test samples
516 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.67200, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 137s - loss: 0.6964 - accuracy: 0.5683 - val_loss: 0.6720 - val_accuracy: 0.5090 - 137s/epoch - 17s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.67200 to 0.63809, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 138s - loss: 0.5652 - accuracy: 0.7410 - val_loss: 0.6381 - val_accuracy: 0.5120 - 138s/epoch - 17s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.63809 to 0.57601, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 138s - loss: 0.4677 - accuracy: 0.7912 - val_loss: 0.5760 - val_accuracy: 0.6175 - 138s/epoch - 17s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.57601 to 0.47461, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 136s - loss: 0.3698 - accuracy: 0.8454 - val_loss: 0.4746 - val_accur

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (996, 64, 2560, 1)
996 train samples
351 test samples
516 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.69718, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 141s - loss: 0.7138 - accuracy: 0.5382 - val_loss: 0.6972 - val_accuracy: 0.5151 - 141s/epoch - 18s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.69718 to 0.66247, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 141s - loss: 0.6119 - accuracy: 0.7068 - val_loss: 0.6625 - val_accuracy: 0.5181 - 141s/epoch - 18s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.66247 to 0.65985, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 138s - loss: 0.5144 - accuracy: 0.8042 - val_loss: 0.6599 - val_accuracy: 0.5151 - 138s/epoch - 17s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.65985 to 0.62874, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 137s - loss: 0.4195 - accuracy: 0.8273 - val_loss: 0.6287 - val_accur

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [6]:
# User split
for one_class in ['HT', 'SNY', 'VIP']:
    model_results = train_model(user_split=True, med_tech_clf=True, data_type='meditation', one_class=one_class)

RUN ID:  1
[0 1] : unique class labels.
X_train shape: (1038, 64, 2560, 1)
1038 train samples
308 test samples
447 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.70100, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 150s - loss: 0.7092 - accuracy: 0.5780 - val_loss: 0.7010 - val_accuracy: 0.5235 - 150s/epoch - 17s/step
Epoch 2/10

Epoch 2: val_loss did not improve from 0.70100
9/9 - 146s - loss: 0.5589 - accuracy: 0.7283 - val_loss: 0.7710 - val_accuracy: 0.6552 - 146s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.70100
9/9 - 141s - loss: 0.3877 - accuracy: 0.8410 - val_loss: 0.8846 - val_accuracy: 0.6614 - 141s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.70100
9/9 - 144s - loss: 0.2568 - accuracy: 0.8979 - val_loss: 1.0227 - val_accuracy: 0.7806 - 144s/epoch - 16s/step
10/10 [==============================] - 1s 92ms/step
EEGNet Classification accuracy test and transfer: 0.444805 0.3

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (1031, 64, 2560, 1)
1031 train samples
325 test samples
460 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.71061, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 148s - loss: 0.7008 - accuracy: 0.5296 - val_loss: 0.7106 - val_accuracy: 0.3269 - 148s/epoch - 16s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.71061 to 0.70330, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 144s - loss: 0.5961 - accuracy: 0.7081 - val_loss: 0.7033 - val_accuracy: 0.4337 - 144s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.70330 to 0.68237, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 145s - loss: 0.4643 - accuracy: 0.8303 - val_loss: 0.6824 - val_accuracy: 0.5469 - 145s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.68237 to 0.62848, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 143s - loss: 0.3041 - accuracy: 0.9195 - val_loss: 0.6285 - val_acc

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1058, 64, 2560, 1)
1058 train samples
311 test samples
485 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.70328, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 145s - loss: 0.7055 - accuracy: 0.5236 - val_loss: 0.7033 - val_accuracy: 0.5270 - 145s/epoch - 16s/step
Epoch 2/10

Epoch 2: val_loss did not improve from 0.70328
9/9 - 146s - loss: 0.6421 - accuracy: 0.6229 - val_loss: 0.7057 - val_accuracy: 0.5743 - 146s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.70328
9/9 - 147s - loss: 0.5138 - accuracy: 0.8185 - val_loss: 0.7042 - val_accuracy: 0.5743 - 147s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.70328 to 0.68508, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 147s - loss: 0.3928 - accuracy: 0.8781 - val_loss: 0.6851 - val_accuracy: 0.6419 - 147s/epoch - 16s/step
Epoch 5/10

Epoch 5: val_loss improved from 0.68508 to 0.65061, saving m

...

[0 1] : unique class labels.
X_train shape: (1046, 64, 2560, 1)
1046 train samples
323 test samples
473 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.74575, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 153s - loss: 0.7070 - accuracy: 0.5793 - val_loss: 0.7457 - val_accuracy: 0.4674 - 153s/epoch - 17s/step
Epoch 2/10

Epoch 2: val_loss did not improve from 0.74575
9/9 - 150s - loss: 0.5884 - accuracy: 0.7046 - val_loss: 0.7769 - val_accuracy: 0.4638 - 150s/epoch - 17s/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.74575
9/9 - 145s - loss: 0.4659 - accuracy: 0.8260 - val_loss: 0.7761 - val_accuracy: 0.4384 - 145s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.74575
9/9 - 147s - loss: 0.3457 - accuracy: 0.8862 - val_loss: 0.7612 - val_accuracy: 0.4601 - 147s/epoch - 16s/step
9/9 [==============================] - 1s 87ms/step
EEGNet Classification accuracy test and transfer: 0.517028 0.665962
RUN ID:

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1026, 64, 2560, 1)
1026 train samples
313 test samples
463 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.68524, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 142s - loss: 0.7173 - accuracy: 0.5214 - val_loss: 0.6852 - val_accuracy: 0.5392 - 142s/epoch - 16s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.68524 to 0.67286, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 148s - loss: 0.6602 - accuracy: 0.6062 - val_loss: 0.6729 - val_accuracy: 0.5000 - 148s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.67286 to 0.67166, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 143s - loss: 0.5998 - accuracy: 0.7057 - val_loss: 0.6717 - val_accuracy: 0.6569 - 143s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.67166
9/9 - 144s - loss: 0.4985 - accuracy: 0.7914 - val_loss: 0.6925 - val_accuracy: 0.6569 - 144s/epoch - 16s/step
Epoch 5/10

Epoch

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1012, 64, 2560, 1)
1012 train samples
320 test samples
482 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.72017, saving model to ./ckpt/checkpoint_user_med_clf.h5
8/8 - 143s - loss: 0.6780 - accuracy: 0.5810 - val_loss: 0.7202 - val_accuracy: 0.4473 - 143s/epoch - 18s/step
Epoch 2/10

Epoch 2: val_loss did not improve from 0.72017
8/8 - 140s - loss: 0.5911 - accuracy: 0.7174 - val_loss: 0.7417 - val_accuracy: 0.4217 - 140s/epoch - 18s/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.72017
8/8 - 144s - loss: 0.4745 - accuracy: 0.8251 - val_loss: 0.7262 - val_accuracy: 0.5272 - 144s/epoch - 18s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.72017 to 0.70156, saving model to ./ckpt/checkpoint_user_med_clf.h5
8/8 - 142s - loss: 0.3344 - accuracy: 0.9121 - val_loss: 0.7016 - val_accuracy: 0.5527 - 142s/epoch - 18s/step
Epoch 5/10


In [5]:
print("Done")

Done
